In [1]:
# Try importing with debug info
import sys
print("Python path:", sys.path)

import llm_utils
print("\nllm_utils location:", llm_utils.__file__)
print("\nContents of llm_utils:", dir(llm_utils))

# Try reloading the module to get latest changes
import importlib
importlib.reload(llm_utils)
print("\nAfter reload - Contents of llm_utils:", dir(llm_utils))

Python path: ['/usr/lib64/python311.zip', '/usr/lib64/python3.11', '/usr/lib64/python3.11/lib-dynload', '', '/proj/sas/ripython/venv_311/lib64/python3.11/site-packages', '/proj/sas/ripython/venv_311/lib/python3.11/site-packages']

llm_utils location: /proj/sas/ripython/LLMClassfication/llm_utils.py

Contents of llm_utils: ['AzureOpenAI', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'ast', 'calculate_disagreement_score', 'calculate_multilabel_metrics', 'cohen_kappa_score', 'convert_gold_labels_to_list', 'create_2shot_hard_prompt', 'generate_explanation_for_easy_example', 'generate_explanation_for_hard_example', 'get_prompt_template', 'get_strengthened_prompt_template', 'json', 'np', 'os', 'parse_llm_labels', 'precision_recall_fscore_support', 're', 'run_llm_annotation', 'run_zero_shot_and_score', 'select_easy_examples_automatically', 'select_hard_examples_automatically', 'time']

After reload - Contents of llm_utils: ['AzureOp

In [1]:
import os
import re
import ast
import pickle
import numpy as np
import pandas as pd
from openai import AzureOpenAI
import time
from sdoh_data import load_sdoh_dataset   # ✅ your helper module
from llm_utils import (parse_llm_labels, run_llm_annotation, calculate_disagreement_score, run_zero_shot_and_score,select_hard_examples_automatically,
                        generate_explanation_for_hard_example, create_2shot_hard_prompt, select_easy_examples_automatically, generate_explanation_for_easy_example,
                        calculate_multilabel_metrics, get_strengthened_prompt_template)
from sklearn.metrics import precision_recall_fscore_support, cohen_kappa_score
import llm_utils

### 📚 Load the saved progress

In [ ]:
# Load saved progress
with open('procgress_checkpoint_step61.pkl', 'rb') as f:
    save_data = pickle.load(f)
# Extract the variables
hard_examples = save_data['hard_examples']
easy_positive = save_data['easy_positive']
easy_negative = save_data['easy_negative']
test_df = save_data['test_df']
results_50 = save_data['results_50']
categories = save_data['categories']

# Verify what we loaded
print("Loaded checkpoint!")
print(f"✓ Hard examples: {len(hard_examples)}")
print(f"✓ Easy positive: {easy_positive['premise'][:50]}...")
print(f"✓ Easy negative: {easy_negative['premise'][:50]}...")
print(f"✓ Test set size: {len(test_df)}")
print()

# Package easy examples as tuple for the function
easy_examples = (easy_positive, easy_negative)

Loaded checkpoint!
✓ Hard examples: 2
✓ Easy positive: The patient smokes about a pack a day for more tha...
✓ Easy negative: She has never used illicit drugs...
✓ Test set size: 200



In [3]:
# Load full evaluation results
with open('full_evaluation_results.pkl', 'rb') as f:
    full_results = pickle.load(f)   
print("Loaded full evaluation results!")
print(f"Configurations evaluated: {list(full_results.keys())}")
print(f"Samples per configuration: {len(full_results['0-Shot'])}")


# sanity check
for config_name, predictions in full_results.items():
    print(f"{config_name}:")
    print(f"  Total predictions: {len(predictions)}")
    print(f"  Sample prediction: {predictions[0]}")

Loaded full evaluation results!
Configurations evaluated: ['0-Shot', '2-Shot Easy', '2-Shot Easy + Exp', '2-Shot Hard', '2-Shot Hard + Exp']
Samples per configuration: 200
0-Shot:
  Total predictions: 200
  Sample prediction: ["employment"]
2-Shot Easy:
  Total predictions: 200
  Sample prediction: []
2-Shot Easy + Exp:
  Total predictions: 200
  Sample prediction: ["employment"]
2-Shot Hard:
  Total predictions: 200
  Sample prediction: []
2-Shot Hard + Exp:
  Total predictions: 200
  Sample prediction: []


In [4]:
# Load your saved progress
with open('all_metrics.pkl', 'rb') as f:
    all_metrics=pickle.load(f) 
all_metrics

{'0-Shot': {'precision_micro': 0.4088669950738916,
  'recall_micro': 0.9764705882352941,
  'f1_micro': 0.5763888888888888,
  'precision_macro': 0.4991309523809523,
  'recall_macro': 0.8523809523809522,
  'f1_macro': 0.5784651918236949,
  'exact_match_accuracy': 0.535,
  'hamming_loss': np.float64(0.061),
  'cohen_kappa': 0.5493914938410682},
 '2-Shot Easy': {'precision_micro': 0.7745098039215687,
  'recall_micro': 0.9294117647058824,
  'f1_micro': 0.8449197860962567,
  'precision_macro': 0.6760714285714284,
  'recall_macro': 0.732034632034632,
  'f1_macro': 0.684804353003303,
  'exact_match_accuracy': 0.865,
  'hamming_loss': np.float64(0.0145),
  'cohen_kappa': 0.8373801379465037},
 '2-Shot Easy + Exp': {'precision_micro': 0.7297297297297297,
  'recall_micro': 0.9529411764705882,
  'f1_micro': 0.826530612244898,
  'precision_macro': 0.6824906245958877,
  'recall_macro': 0.7493506493506492,
  'f1_macro': 0.6927342516638292,
  'exact_match_accuracy': 0.845,
  'hamming_loss': np.float64(

In [5]:
with open('final_best_model_f1_094.pkl', 'rb') as f:
    final_best_model=pickle.load(f)
print("✅ Final model saved to: final_best_model_f1_094.pkl")
print()
print("Final Performance:")
print(f"  F1 (micro):     {final_best_model['f1_micro']:.3f}")
print(f"  F1 (macro):     {final_best_model['f1_macro']:.3f}")
print(f"  Precision:      {final_best_model['precision_micro']:.3f}")
print(f"  Recall:         {final_best_model['recall_micro']:.3f}")
print(f"  Cohen's Kappa:  {final_best_model['cohen_kappa']:.3f}")
print(f"  Exact Match:    {final_best_model['exact_match']:.3f}")
print(f"  Accuracy:       {final_best_model['accuracy']:.1%}")
print(f"  Errors:         {final_best_model['errors']}/200")
print()

✅ Final model saved to: final_best_model_f1_094.pkl

Final Performance:
  F1 (micro):     0.940
  F1 (macro):     0.648
  Precision:      0.952
  Recall:         0.929
  Cohen's Kappa:  0.938
  Exact Match:    0.950
  Accuracy:       95.0%
  Errors:         10/200



### 🚀 XGBoost Implementation Plan<br>
Phase 1: Label 320 Unused Samples with GPT-4o-mini v3

In [6]:
# Use the same 2-Shot Easy prompt that worked best for GPT-3.5
examples_easy_no_exp = f"""
Example 1 (Positive):
Premise: "{easy_positive['premise']}"
Answer: {parse_llm_labels(easy_positive['gold_labels'])}

Example 2 (Negative):
Premise: "{easy_negative['premise']}"
Answer: {parse_llm_labels(easy_negative['gold_labels'])}
"""
original_template = llm_utils.get_prompt_template

In [7]:
print("="*70)
print("PHASE 1: LABELING 320 UNUSED SAMPLES WITH GPT-4o-mini v3")
print("="*70)
print()

# Get the 320 unused samples
# We used 50 for hard example selection + 200 for testing = 250 used
# Total 570 - 250 = 320 remaining

# Load original data
# from sdoh_data import load_sdoh_dataset
# df_full = load_sdoh_dataset()
df_gold = pd.read_csv("annotations_with_gold.csv")
print(f"Full dataset size: {len(df_gold)}")
print()

# Identify unused samples (those not in results_50 or test_df)
used_indices = set(results_50.index.tolist() + test_df.index.tolist())
unused_mask = ~df_gold.index.isin(used_indices)
df_unused = df_gold[unused_mask].copy()

print(f"Unused samples: {len(df_unused)}")
print(f"Used samples: {len(used_indices)}")
print(f"Total: {len(df_gold)}")
print()

# Verify we have the right amount
assert len(df_unused) == 320, f"Expected 320 unused samples, got {len(df_unused)}"

print("✓ Verified: 320 unused samples ready for labeling")
print()

# Use same strengthened prompt that achieved F1=0.940
llm_utils.get_prompt_template = get_strengthened_prompt_template

print("Labeling 320 samples with GPT-4o-mini v3 (strengthened prompt)...")
print("Estimated time: ~5-7 minutes")
print("Estimated cost: ~$0.50")
print()

# Label with GPT-4o-mini v3
xgb_training_labels = run_llm_annotation(
    df=df_unused,
    categories=categories,
    examples=examples_easy_no_exp,  # Same 2-shot easy examples
    deployment_name="gpt-4o-mini",
    sleep_time=1
)

print(f"✓ Got {len(xgb_training_labels)} labels!")
print()

# Parse labels
xgb_training_labels_parsed = [parse_llm_labels(label) for label in xgb_training_labels]

print("Sample of labeled data:")
for i in range(3):
    print(f"  {i+1}. {df_unused.iloc[i]['premise'][:60]}...")
    print(f"     Labels: {xgb_training_labels_parsed[i]}")
print()

# Save labeled training data
df_unused['gpt4o_labels'] = xgb_training_labels
df_unused['gpt4o_labels_parsed'] = xgb_training_labels_parsed

# Restore template
llm_utils.get_prompt_template = original_template

print("✓ Phase 1 complete: 320 samples labeled with GPT-4o-mini v3")
print()

PHASE 1: LABELING 320 UNUSED SAMPLES WITH GPT-4o-mini v3

Full dataset size: 570

Unused samples: 320
Used samples: 250
Total: 570

✓ Verified: 320 unused samples ready for labeling

Labeling 320 samples with GPT-4o-mini v3 (strengthened prompt)...
Estimated time: ~5-7 minutes
Estimated cost: ~$0.50

✓ Got 320 labels!

Sample of labeled data:
  1. 1-2 ppd Cigarettes...
     Labels: ['smoking']
  2. 2ppd smoker since his teens; quit 2 years ago...
     Labels: ['smoking']
  3. According to the chart, the patient also drinks wine everyda...
     Labels: ['alcohol', 'drug_use']

✓ Phase 1 complete: 320 samples labeled with GPT-4o-mini v3



The above drug_use false alarm again in Sample 3. but it is consistent with the 4FP errors we saw earlier. GPT4o min occationally over apply the drug umbrella rule to alcohol <br>
**But thats OK** XGBoost will learn from the aggregate samples across 320 samples and will correct these errors. Thats the beauty of statiscial learning!🎯<br>

In [8]:
print("="*70)
print("CHECKING GPT-4o-mini LABELING QUALITY ON 320 SAMPLES")
print("="*70)
print()
# Get gold labels for the 320 unused samples
# These should already exist in df_unused from the original dataset
df_gold['gold_labels_parsed'] = df_gold['gold_labels'].apply(parse_llm_labels)
df_unused = df_gold[unused_mask].copy()
print(f"Extracted gold labels for {len(df_unused)} samples")
print()

CHECKING GPT-4o-mini LABELING QUALITY ON 320 SAMPLES

Extracted gold labels for 320 samples



In [9]:
gold_labels_320=[label for label in df_unused['gold_labels_parsed']] 

# Calculate metrics: GPT-4o-mini vs Gold on 320 samples
metrics_gpt4o_on_320 = calculate_multilabel_metrics(
    gold_labels=gold_labels_320,
    predicted_labels=xgb_training_labels_parsed,
    categories=categories)

print("GPT-4o-mini Performance on 320 Training Samples:")
print(f"  F1 (micro):     {metrics_gpt4o_on_320['f1_micro']:.3f}")
print(f"  Cohen's Kappa:  {metrics_gpt4o_on_320['cohen_kappa']:.3f}")
print(f"  Exact Match:    {metrics_gpt4o_on_320['exact_match_accuracy']:.3f}")
print()


GPT-4o-mini Performance on 320 Training Samples:
  F1 (micro):     0.882
  Cohen's Kappa:  0.876
  Exact Match:    0.906



In [21]:
# Analyze errors in training data
errors_in_training = []
for i in range(len(gold_labels_320)):
    if set(gold_labels_320[i]) != set(xgb_training_labels_parsed[i]):
        errors_in_training.append(i)

print(f"Errors in training labels: {len(errors_in_training)}/{len(gold_labels_320)} ({len(errors_in_training)/len(gold_labels_320)*100:.1f}%)")
print()

Errors in training labels: 30/320 (9.4%)



🚀 Phase 2: Feature Engineering & XGBoost Training

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
import numpy as np

print("="*70)
print("PHASE 2: FEATURE ENGINEERING & XGBOOST TRAINING")
print("="*70)
print()

# Install xgboost if needed
try:
    import xgboost as xgb
    print("✓ XGBoost already installed")
except ImportError:
    print("Installing XGBoost...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'xgboost'])
    import xgboost as xgb
    print("✓ XGBoost installed")


PHASE 2: FEATURE ENGINEERING & XGBOOST TRAINING

✓ XGBoost already installed


In [15]:

print()
print("Step 1: Prepare Training Data")
print("-" * 50)

# Training data: 320 samples labeled by GPT-4o-mini
X_train_text = df_unused['premise'].values
y_train_labels = xgb_training_labels_parsed

# Test data: 200 samples with gold labels
X_test_text = test_df['premise'].values
gold_labels_parsed=[
    parse_llm_labels(label) for label in test_df['gold_labels']
]
y_test_labels = gold_labels_parsed

print(f"Training samples: {len(X_train_text)}")
print(f"Test samples: {len(X_test_text)}")
print()

# Step 2: Convert labels to binary matrix
print("Step 2: Convert Labels to Binary Format")
print("-" * 50)

mlb = MultiLabelBinarizer(classes=categories)
mlb.fit([categories])  # Fit on all categories

y_train_binary = mlb.transform(y_train_labels)
y_test_binary = mlb.transform(y_test_labels)

print(f"Binary matrix shape: {y_train_binary.shape}")
print(f"Categories: {mlb.classes_}")
print()

# Check label distribution in training data
print("Training label distribution:")
for i, cat in enumerate(categories):
    count = y_train_binary[:, i].sum()
    print(f"  {cat:15s}: {count:3d} ({count/len(y_train_binary)*100:.1f}%)")
print()

# Step 3: Extract TF-IDF features
print("Step 3: Extract TF-IDF Features")
print("-" * 50)
# TF-IDF with n-grams (following paper's approach)
tfidf = TfidfVectorizer(
    max_features=1000,      # Top 1000 features
    ngram_range=(1, 2),     # Unigrams and bigrams
    min_df=2,               # Minimum document frequency
    max_df=0.8,             # Maximum document frequency
    stop_words='english',   # Remove common English words
    lowercase=True
)


Step 1: Prepare Training Data
--------------------------------------------------
Training samples: 320
Test samples: 200

Step 2: Convert Labels to Binary Format
--------------------------------------------------
Binary matrix shape: (320, 10)
Categories: ['food' 'opioids' 'cocaine' 'smoking' 'marijuana' 'transportation'
 'housing' 'employment' 'drug_use' 'alcohol']

Training label distribution:
  food           :   0 (0.0%)
  opioids        :   2 (0.6%)
  cocaine        :   2 (0.6%)
  smoking        :  53 (16.6%)
  marijuana      :   4 (1.2%)
  transportation :   0 (0.0%)
  housing        :   0 (0.0%)
  employment     :  10 (3.1%)
  drug_use       :  30 (9.4%)
  alcohol        :  49 (15.3%)

Step 3: Extract TF-IDF Features
--------------------------------------------------


In [16]:
# Fit on training data
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)
print(f"TF-IDF matrix shape: {X_train_tfidf.shape}")
print(f"Vocabulary size: {len(tfidf.vocabulary_)}")
# Show some top features
feature_names = tfidf.get_feature_names_out()
print("Sample features learned:")
print(f"  {feature_names[:20]}")
print()

TF-IDF matrix shape: (320, 308)
Vocabulary size: 308
Sample features learned:
  ['10' '10 years' '11' '15' '15 years' '1968' '20' '20 pack' '20 years'
 '25' '27' '2ppd' '30' '30 years' '40' '40 years' '50' '50 years' 'abc'
 'abuse']



In [17]:
# Nothing wrong below syntax wise, but there are certain categories have 0 samples in training data(food, transportation, housing)
#  XGBoost can't train a binary classifier when there are no positive examples!
# Step 4: Train XGBoost (one classifier per category)
print("Step 4: Train XGBoost Classifiers")
print("-" * 50)
print("Training multi-label XGBoost (10 binary classifiers)...")
print()

# XGBoost parameters (following paper's approach)
xgb_params = {
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'use_label_encoder': False,
    'base_score': 0.5,  # any float between 0 and 1
    'random_state': 42
}

# Train one classifier per category (multi-label approach)
xgb_classifiers = MultiOutputClassifier(
    xgb.XGBClassifier(**xgb_params)
)

print("Training...")
xgb_classifiers.fit(X_train_tfidf, y_train_binary)
print("✓ Training complete!")
print()

print("="*70)
print("PHASE 2 COMPLETE: XGBoost Model Trained!")
print("="*70)
print()

Step 4: Train XGBoost Classifiers
--------------------------------------------------
Training multi-label XGBoost (10 binary classifiers)...

Training...


/proj/sas/ripython/venv_311/lib64/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:27:04] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/proj/sas/ripython/venv_311/lib64/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:27:07] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/proj/sas/ripython/venv_311/lib64/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:27:11] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/proj/sas/ripython/venv_311/lib64/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:27:14] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/proj/sas/ripython/venv_

✓ Training complete!

PHASE 2 COMPLETE: XGBoost Model Trained!



In [19]:
print("======================================================================")
print("PHASE 3: EVALUATING XGBOOST ON 200 TEST SAMPLES")
print("======================================================================")
print()

# Make predictions
print("Making predictions with XGBoost...")
y_pred_binary = xgb_classifiers.predict(X_test_tfidf)

print(f"✓ Got predictions! Shape: {y_pred_binary.shape}")
print()

# Convert binary predictions back to label lists
y_pred_labels = mlb.inverse_transform(y_pred_binary)
y_pred_labels = [list(labels) for labels in y_pred_labels]

# Show first 3 predictions
print("Sample predictions (first 3):")
for i in range(3):
    print(f"  {i+1}. Text: {X_test_text[i][:60]}...")
    print(f"     Gold:      {y_test_labels[i]}")
    print(f"     Predicted: {y_pred_labels[i]}")
    print()

PHASE 3: EVALUATING XGBOOST ON 200 TEST SAMPLES

Making predictions with XGBoost...
✓ Got predictions! Shape: (200, 10)

Sample predictions (first 3):
  1. Text: Semi-retired Attorney...
     Gold:      []
     Predicted: []

  2. Text: He does not smoke, occasionally drinks alcohol...
     Gold:      ['alcohol']
     Predicted: ['drug_use', 'alcohol']

  3. Text: A 76-year-old who used to smoke a pack a day and quit in 198...
     Gold:      ['smoking']
     Predicted: ['smoking']



In [20]:
# Calculate metrics for XGBoost
y_pred_parsed_labels=[parse_llm_labels(label) for label in y_pred_labels]
xgb_metrics = calculate_multilabel_metrics(
    gold_labels=y_test_labels,
    predicted_labels=y_pred_parsed_labels,
    categories=categories
)

print("XGBoost Performance on 200 Test Samples:")
print(f"  F1 (micro):     {xgb_metrics['f1_micro']:.3f}")
print(f"  Precision:      {xgb_metrics['precision_micro']:.3f}")
print(f"  Recall:         {xgb_metrics['recall_micro']:.3f}")
print(f"  Cohen's Kappa:  {xgb_metrics['cohen_kappa']:.3f}")
print(f"  Exact Match:    {xgb_metrics['exact_match_accuracy']:.3f}")
print()

# Compare to GPT-4o-mini best result
print("Comparison:")
print(f"  GPT-4o-mini (best): F1 = {final_best_model['f1_micro']:.3f}")
print(f"  XGBoost:            F1 = {xgb_metrics['f1_micro']:.3f}")

XGBoost Performance on 200 Test Samples:
  F1 (micro):     0.560
  Precision:      0.646
  Recall:         0.494
  Cohen's Kappa:  0.543
  Exact Match:    0.755

Comparison:
  GPT-4o-mini (best): F1 = 0.940
  XGBoost:            F1 = 0.560


In [98]:
print("Per-category Performance:")
print()

# Get per-category metrics
for i, cat in enumerate(categories):
    # Extract this category's predictions
    y_true_cat = y_test_binary[:, i]
    y_pred_cat = y_pred_binary[:, i]
    
    tp = ((y_true_cat == 1) & (y_pred_cat == 1)).sum()
    fp = ((y_true_cat == 0) & (y_pred_cat == 1)).sum()
    fn = ((y_true_cat == 1) & (y_pred_cat == 0)).sum()
    
    if tp + fp > 0:
        precision = tp / (tp + fp)
    else:
        precision = 0
        
    if tp + fn > 0:
        recall = tp / (tp + fn)
    else:
        recall = 0
        
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = 0
    
    print(f"{cat:15s}: F1={f1:.3f}  P={precision:.3f}  R={recall:.3f}  (Gold={y_true_cat.sum()}, Pred={y_pred_cat.sum()})")

# XGBoost is only predicting 3 categories well:

# ✅ smoking: F1=0.793 (33 gold samples - enough data!)
# ⚠️ alcohol: F1=0.600 (33 gold samples)
# ❌ employment: F1=0.286 (only 3 gold samples)

# XGBoost predicts NOTHING for rare categories:

# opioids, cocaine, marijuana, transportation, housing = 0 predictions
# drug_use = predicts 10 but all wrong (F1=0.000)

# The root cause: XGBoost trained on 320 samples where these rare categories had almost no examples!

Per-category Performance:

food           : F1=0.000  P=0.000  R=0.000  (Gold=0, Pred=0)
opioids        : F1=0.000  P=0.000  R=0.000  (Gold=2, Pred=0)
cocaine        : F1=0.000  P=0.000  R=0.000  (Gold=2, Pred=0)
smoking        : F1=0.793  P=0.920  R=0.697  (Gold=33, Pred=25)
marijuana      : F1=0.000  P=0.000  R=0.000  (Gold=3, Pred=0)
transportation : F1=0.000  P=0.000  R=0.000  (Gold=1, Pred=0)
housing        : F1=0.000  P=0.000  R=0.000  (Gold=1, Pred=0)
employment     : F1=0.286  P=0.250  R=0.333  (Gold=3, Pred=4)
drug_use       : F1=0.000  P=0.000  R=0.000  (Gold=7, Pred=10)
alcohol        : F1=0.600  P=0.667  R=0.545  (Gold=33, Pred=27)


In [86]:
# We need a hybrid approach: Use XGBoost for common categories, GPT-4o-mini for rare ones.
print("Creating Hybrid Model: XGBoost + GPT-4o-mini")
print("="*70)
print()

# Categories with good XGBoost performance (F1 > 0.5)
xgb_categories = ['smoking', 'alcohol']

# Categories to use GPT-4o-mini for
gpt_categories = ['food', 'opioids', 'cocaine', 'marijuana', 
                  'transportation', 'housing', 'employment', 'drug_use']

print(f"XGBoost handles: {xgb_categories}")
print(f"GPT-4o-mini handles: {gpt_categories}")
print()

Creating Hybrid Model: XGBoost + GPT-4o-mini

XGBoost handles: ['smoking', 'alcohol']
GPT-4o-mini handles: ['food', 'opioids', 'cocaine', 'marijuana', 'transportation', 'housing', 'employment', 'drug_use']



In [24]:
print("Building Hybrid Predictions...")
print()

# We need GPT-4o-mini predictions on test set
# Do you have them already? Let me check...

# From your artifacts, you have: full_results['2-Shot Easy']
gpt_predictions = full_results['2-Shot Easy']

print(f"✓ Got {len(gpt_predictions)} GPT-4o-mini predictions")
print()


Building Hybrid Predictions...

✓ Got 200 GPT-4o-mini predictions



In [25]:
# Create hybrid predictions
hybrid_predictions = []

for i in range(len(y_test_labels)):
    # Start with GPT-4o-mini prediction
    gpt_pred = parse_llm_labels(gpt_predictions[i]) if gpt_predictions[i] else []
    
    # Get XGBoost prediction for this sample
    xgb_pred = y_pred_labels[i]
    
    # Combine: Use XGBoost for smoking/alcohol, GPT for everything else
    hybrid = []
    
    # Add XGBoost predictions for smoking and alcohol
    if 'smoking' in xgb_pred:
        hybrid.append('smoking')
    if 'alcohol' in xgb_pred:
        hybrid.append('alcohol')
    
    # Add GPT predictions for all other categories
    for label in gpt_pred:
        if label not in ['smoking', 'alcohol'] and label not in hybrid:
            hybrid.append(label)
    
    hybrid_predictions.append(hybrid)

# Show first 3
print("Sample hybrid predictions:")
for i in range(3):
    print(f"  {i+1}. Gold: {y_test_labels[i]}")
    print(f"     GPT:  {gpt_predictions[i]}")
    print(f"     XGB:  {y_pred_labels[i]}")
    print(f"     Hybrid: {hybrid_predictions[i]}")
    print()

Sample hybrid predictions:
  1. Gold: []
     GPT:  []
     XGB:  []
     Hybrid: []

  2. Gold: ['alcohol']
     GPT:  ["alcohol"]
     XGB:  ['drug_use', 'alcohol']
     Hybrid: ['alcohol']

  3. Gold: ['smoking']
     GPT:  ["smoking"]
     XGB:  ['smoking']
     Hybrid: ['smoking']



In [26]:
# Calculate metrics for XGBoost
xgb_metrics = calculate_multilabel_metrics(
    gold_labels=y_test_labels,
    predicted_labels=hybrid_predictions,
    categories=categories
)

print("XGBoost Performance on 200 Test Samples:")
print(f"  F1 (micro):     {xgb_metrics['f1_micro']:.3f}")
print(f"  Precision:      {xgb_metrics['precision_micro']:.3f}")
print(f"  Recall:         {xgb_metrics['recall_micro']:.3f}")
print(f"  Cohen's Kappa:  {xgb_metrics['cohen_kappa']:.3f}")
print(f"  Exact Match:    {xgb_metrics['exact_match_accuracy']:.3f}")
print()

# Compare to GPT-4o-mini best result
print("Comparison:")
print(f"  GPT-4o-mini (best): F1 = {final_best_model['f1_micro']:.3f}")
print(f"  XGBoost:            F1 = {xgb_metrics['f1_micro']:.3f}")

XGBoost Performance on 200 Test Samples:
  F1 (micro):     0.659
  Precision:      0.671
  Recall:         0.647
  Cohen's Kappa:  0.644
  Exact Match:    0.730

Comparison:
  GPT-4o-mini (best): F1 = 0.940
  XGBoost:            F1 = 0.659


In [27]:
print("Analyzing the Training Data Quality Issue:")
print("="*70)
print()

print("Training Data:")
print(f"  Samples: 320")
print(f"  Labeled by: GPT-4o-mini")
print(f"  Quality: F1 = {metrics_gpt4o_on_320['f1_micro']:.3f} (10% error rate)")
print()

print("The Problem:")
print("  XGBoost learned from noisy labels with 10% errors")
print("  It amplified these errors, especially for rare categories")
print()

print("Why GPT-4o-mini is better:")
print(f"  GPT-4o-mini direct:  F1 = 0.940")
print(f"  XGBoost (learned):   F1 = 0.556")  
print(f"  Hybrid approach:     F1 = 0.667")
print()

print("Conclusion:")
print("  ❌ XGBoost didn't help - it made things worse!")
print("  ✅ Stick with GPT-4o-mini (F1 = 0.940)")

Analyzing the Training Data Quality Issue:

Training Data:
  Samples: 320
  Labeled by: GPT-4o-mini
  Quality: F1 = 0.882 (10% error rate)

The Problem:
  XGBoost learned from noisy labels with 10% errors
  It amplified these errors, especially for rare categories

Why GPT-4o-mini is better:
  GPT-4o-mini direct:  F1 = 0.940
  XGBoost (learned):   F1 = 0.556
  Hybrid approach:     F1 = 0.667

Conclusion:
  ❌ XGBoost didn't help - it made things worse!
  ✅ Stick with GPT-4o-mini (F1 = 0.940)


#### Test with another option to see if XGBoost performance can be improved

In [28]:
mlb = MultiLabelBinarizer(classes=categories)
mlb.fit([categories])  # Fit on all categories
y_test=[parse_llm_labels(label) for label in test_df['gold_labels']]
y_test_binary=mlb.transform(y_test)

In [29]:
print("="*70)
print("OPTION B: TRAINING XGBOOST ON GOLD LABELS (200 SAMPLES)")
print("="*70)
print()

print("Strategy: Use the 200 test samples with gold labels for training")
print("Problem: We need a separate test set to evaluate!")
print()

# Split the 200 samples: 150 train, 50 test
from sklearn.model_selection import train_test_split

print("Splitting 200 gold-labeled samples:")
print("  Training: 150 samples")
print("  Testing:  50 samples")
print()
 
# Split
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(
    test_df['premise'].values,
    y_test_binary,
    test_size=50,
    random_state=42
)

print(f"✓ Training set: {len(X_train_gold)} samples")
print(f"✓ Test set: {len(X_test_gold)} samples")
print()

# Extract TF-IDF features
print("Extracting TF-IDF features...")
tfidf_gold = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8,
    stop_words='english',
    lowercase=True
)

X_train_gold_tfidf = tfidf_gold.fit_transform(X_train_gold)
X_test_gold_tfidf = tfidf_gold.transform(X_test_gold)

print(f"✓ TF-IDF shape: {X_train_gold_tfidf.shape}")
print()


OPTION B: TRAINING XGBOOST ON GOLD LABELS (200 SAMPLES)

Strategy: Use the 200 test samples with gold labels for training
Problem: We need a separate test set to evaluate!

Splitting 200 gold-labeled samples:
  Training: 150 samples
  Testing:  50 samples

✓ Training set: 150 samples
✓ Test set: 50 samples

Extracting TF-IDF features...
✓ TF-IDF shape: (150, 143)



In [30]:
print("Training XGBoost on 150 gold-labeled samples...")
print()
xgb_gold=MultiOutputClassifier(xgb.XGBClassifier(**xgb_params))
print("Training...")
xgb_gold.fit(X_train_gold_tfidf, y_train_gold)
print("✓ Training complete!")
print()
print("Making predictions with XGBoost...")
y_pred_gold_binary=xgb_gold.predict(X_test_gold_tfidf)
print(f"✓ Got predictions! Shape: {y_pred_gold_binary.shape}")
print() 
# convert binary back to label lists
y_pred_gold_labels=mlb.inverse_transform(y_pred_gold_binary)
y_pred_gold_labels=[list(labels) for labels in y_pred_gold_labels]

y_test_gold_labels = mlb.inverse_transform(y_test_gold)
y_test_gold_labels = [list(labels) for labels in y_test_gold_labels]


Training XGBoost on 150 gold-labeled samples...

Training...


/proj/sas/ripython/venv_311/lib64/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:43:38] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/proj/sas/ripython/venv_311/lib64/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:43:40] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/proj/sas/ripython/venv_311/lib64/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:43:44] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/proj/sas/ripython/venv_311/lib64/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:43:47] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/proj/sas/ripython/venv_

✓ Training complete!

Making predictions with XGBoost...
✓ Got predictions! Shape: (50, 10)



In [38]:
# Show first 3 predictions
print("Sample predictions (first 3):")
for i in range(3):
    print(f"  {i+1}. Text: {X_test_gold[i][:60]}...")
    print(f"     Gold:      {y_test_gold_labels[i]}")
    print(f"     Predicted: {y_pred_gold_labels[i]}")
    print()

Sample predictions (first 3):
  1. Text: She is a student...
     Gold:      []
     Predicted: []

  2. Text: She lives with a room-mate....
     Gold:      []
     Predicted: []

  3. Text: Occasional glass of wine at dinner...
     Gold:      ['alcohol']
     Predicted: []



In [39]:
# Calculate metrics for XGBoost 

metrics_gold = calculate_multilabel_metrics(
    gold_labels=y_test_gold_labels,
    predicted_labels=y_pred_gold_labels,
    categories=categories
)

print("XGBoost trained on GOLD labels (150 samples):")
print(f"  F1 (micro):     {metrics_gold['f1_micro']:.3f}")
print(f"  Precision:      {metrics_gold['precision_micro']:.3f}")
print(f"  Recall:         {metrics_gold['recall_micro']:.3f}")
print(f"  Cohen's Kappa:  {metrics_gold['cohen_kappa']:.3f}")
print(f"  Exact Match:    {metrics_gold['exact_match_accuracy']:.3f}")
print()
print("Comparison:")
print(f"  XGBoost on noisy (320): F1 = 0.556")
print(f"  XGBoost on gold (150):  F1 = {metrics_gold['f1_micro']:.3f}")
print(f"  GPT-4o-mini direct:     F1 = 0.940")


XGBoost trained on GOLD labels (150 samples):
  F1 (micro):     0.480
  Precision:      0.667
  Recall:         0.375
  Cohen's Kappa:  0.468
  Exact Match:    0.780

Comparison:
  XGBoost on noisy (320): F1 = 0.556
  XGBoost on gold (150):  F1 = 0.480
  GPT-4o-mini direct:     F1 = 0.940


🚀 Option C: Confidence-Based Hybrid. The idea: Use XGBoost's prediction probabilities instead of hard predictions. Only trust XGBoost when it's very confident!

In [40]:
print("="*70)
print("OPTION C: CONFIDENCE-BASED HYBRID")
print("="*70)
print()

print("Strategy: Use prediction confidence scores")
print("  - If XGBoost is VERY confident (>0.8), use XGBoost")
print("  - Otherwise, use GPT-4o-mini")
print()

# Get XGBoost probability predictions (not just 0/1)
print("Getting XGBoost confidence scores...")
xgb_proba = xgb_classifiers.predict_proba(X_test_tfidf)

print(f"✓ Got probability predictions")
print(f"  Shape: {len(xgb_proba)} classifiers")
print()

# Show sample probabilities for first sample
print("Sample probabilities for first test sample:")
for i, cat in enumerate(categories):
    # Each classifier returns [prob_negative, prob_positive]
    prob_positive = xgb_proba[i][0][1]  # Probability of class=1
    print(f"  {cat:15s}: {prob_positive:.3f}")

OPTION C: CONFIDENCE-BASED HYBRID

Strategy: Use prediction confidence scores
  - If XGBoost is VERY confident (>0.8), use XGBoost
  - Otherwise, use GPT-4o-mini

Getting XGBoost confidence scores...
✓ Got probability predictions
  Shape: 10 classifiers

Sample probabilities for first test sample:
  food           : 0.003
  opioids        : 0.003
  cocaine        : 0.004
  smoking        : 0.012
  marijuana      : 0.002
  transportation : 0.003
  housing        : 0.003
  employment     : 0.014
  drug_use       : 0.025
  alcohol        : 0.032


In [41]:
print("Analyzing XGBoost confidence across ALL 200 samples:")
print()

for j, cat in enumerate(categories):
    # Get probabilities for this category across all 200 samples
    all_probs = xgb_proba[j][:, 1]  # All samples, positive class
    
    max_prob = all_probs.max()
    min_prob = all_probs.min()
    avg_prob = all_probs.mean()
    
    # Count how many samples have high confidence (>0.5)
    high_conf_count = (all_probs > 0.5).sum()
    
    print(f"{cat:15s}: max={max_prob:.3f}, avg={avg_prob:.3f}, min={min_prob:.3f}, >0.5: {high_conf_count}/200")

print()
print("This shows us if XGBoost is confident about ANY samples!")

Analyzing XGBoost confidence across ALL 200 samples:

food           : max=0.003, avg=0.003, min=0.003, >0.5: 0/200
opioids        : max=0.063, avg=0.006, min=0.003, >0.5: 0/200
cocaine        : max=0.026, avg=0.005, min=0.004, >0.5: 0/200
smoking        : max=0.990, avg=0.130, min=0.004, >0.5: 24/200
marijuana      : max=0.344, avg=0.016, min=0.001, >0.5: 0/200
transportation : max=0.003, avg=0.003, min=0.003, >0.5: 0/200
housing        : max=0.003, avg=0.003, min=0.003, >0.5: 0/200
employment     : max=0.669, avg=0.029, min=0.006, >0.5: 4/200
drug_use       : max=0.924, avg=0.088, min=0.002, >0.5: 10/200
alcohol        : max=0.991, avg=0.168, min=0.004, >0.5: 27/200

This shows us if XGBoost is confident about ANY samples!


In [42]:
print("="*70)
print("SMART HYBRID: XGBoost (when confident) + GPT-4o-mini")
print("="*70)
print()

print("Rules:")
print("  - smoking/alcohol: Use XGBoost if confidence > 0.5")
print("  - Everything else: Use GPT-4o-mini")
print()

# Create smart hybrid predictions
smart_hybrid = []

for i in range(len(y_test_labels)):
    # Start with GPT prediction
    gpt_pred = parse_llm_labels(gpt_predictions[i]) if gpt_predictions[i] else []
    prediction = list(gpt_pred)
    
    # Override smoking if XGBoost is confident
    smoking_idx = categories.index('smoking')
    if xgb_proba[smoking_idx][i][1] > 0.5:
        if 'smoking' not in prediction:
            prediction.append('smoking')
    elif xgb_proba[smoking_idx][i][1] < 0.5:
        if 'smoking' in prediction:
            prediction.remove('smoking')
    
    # Override alcohol if XGBoost is confident
    alcohol_idx = categories.index('alcohol')
    if xgb_proba[alcohol_idx][i][1] > 0.5:
        if 'alcohol' not in prediction:
            prediction.append('alcohol')
    elif xgb_proba[alcohol_idx][i][1] < 0.5:
        if 'alcohol' in prediction:
            prediction.remove('alcohol')
    
    smart_hybrid.append(prediction)

# Calculate metrics
metrics_smart = calculate_multilabel_metrics(
    gold_labels=y_test_labels,
    predicted_labels=smart_hybrid,
    categories=categories
)

print("Results:")
print(f"  Smart Hybrid: F1 = {metrics_smart['f1_micro']:.3f}")
print()
print("Comparison:")
print(f"  GPT-4o-mini alone:  F1 = 0.940")
print(f"  XGBoost alone:      F1 = 0.564")
print(f"  Smart Hybrid:       F1 = {metrics_smart['f1_micro']:.3f}")

SMART HYBRID: XGBoost (when confident) + GPT-4o-mini

Rules:
  - smoking/alcohol: Use XGBoost if confidence > 0.5
  - Everything else: Use GPT-4o-mini

Results:
  Smart Hybrid: F1 = 0.659

Comparison:
  GPT-4o-mini alone:  F1 = 0.940
  XGBoost alone:      F1 = 0.564
  Smart Hybrid:       F1 = 0.659


### XGBoost Failed:
***Root Causes***:
- Small dataset: 320 training samples insufficient for 10 categories
- Severe class imbalance: 6/10 categories had <5 training examples
- Noisy training labels: GPT-4o-mini labels had 10-12% error rate (F1=0.88-0.90)
- Variability in LLM labels: Running GPT-4o-mini twice on same 320 samples gave F1=0.90 then F1=0.88,<br>

***Comparison to Paper (arxiv 2407.17126)***:<br>
The Paper:<br>
- Used LLM → XGBoost approach (same as you!) 
- Likely had 1000s of training samples (not 320) 
- Better class balance 
- Achieved 0.90+ AUROC 

***Your Work:***<br>
- Only 320 samples
- 6 categories with <5 examples each
- XGBoost couldn't learn rare categories

***Conclusion: Your approach was correct; you just need more data to make XGBoost competitive!<br>***

***FINAL RESULTS COMPARISON:<br>***
Model|                          F1 Score
════════════════════════════════════════
GPT-4o-mini (direct)           0.940 ✅ WINNER
GPT-3.5 Turbo 2-Shot Easy      0.845 (baseline)
XGBoost on noisy (320)         0.564
Hybrid (simple)                0.667
Smart Hybrid (confidence)      0.659
XGBoost on gold (150)          0.480

***KEY TECHNICAL DETAILS***
***Data Split:***

- 570 total samples
- 50 used for hard example selection
- 200 used for testing (gold labels)
- 320 unused samples (used for XGBoost training)

***Categories:***
food, opioids, cocaine, smoking, marijuana, transportation, housing, employment, drug_use, alcohol

In [ ]:
# XGBoost Configuration:
xgb_params = {
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'random_state': 42
}
# TF-IDF Features:
TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8,
    stop_words='english',
    lowercase=True
) 
## **CODE ARTIFACTS YOU HAVE**

1. `llm_utils.py` - All helper functions
2. `sdoh_data.py` - Data loading
3. Hard example selection code
4. Easy example selection code
5. Prompt engineering evaluation code
6. GPT-4o-mini strengthened prompt
7. XGBoost training & evaluation code
8. Hybrid model code
9. Confidence analysis code